## 1️⃣ Prerequisites

Make sure your system has:

 - Python ≥ 3.9
 - pip (latest)
 - Jupyter Lab / Notebook
 - Git installed
 - An active Google API key
```bash
python -m venv .venv
.venv\Scripts\Activate.ps1

python3 -m venv .venv
source .venv/bin/activate # mak/Linux

```
## 2️⃣ Set Up a .env File for Secrets

Create a file named .env in the root directory of your notebook project:
```bash
touch .env
```

Then open .env and add your API key:

```
GOOGLE_API_KEY=your_real_api_key_here
GOOGLE_GENAI_USE_VERTEXAI=FALSE
``` 


## 3️⃣ Load Environment Variables in Jupyter

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Environment ready – Gemini API key loaded.")
else:
    raise EnvironmentError("🔑 GOOGLE_API_KEY not found. Please create a .env file.")


✅ Environment ready – Gemini API key loaded.


## 4️⃣ Prepare for Git Version Control
 - Add a .gitignore file in your project root (or update it):

```
# Python
.venv/
__pycache__/
.ipynb_checkpoints/

# Environment / secrets
.env

# OS & IDE files
.DS_Store
Thumbs.db
.vscode/
```

## 5️⃣ Then initialize and commit safely:
```
    git init
    git add .
    git commit -m "Initial commit – reproducible Jupyter ADK environment"
```

## 6️⃣ Recreate & Launch Your Environment
    - use requirements.txt to relunch the environment 
```bash
    pip install -r requirements.txt
```

## 8️⃣ Create & use a virtual environment + register Jupyter kernel

- **Windows (PowerShell):**

```bash
python3 -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
python -m ipykernel install --user --name "adk-env" --display-name "Python (adk-env)"
```
- **macOS/Linux:**
```bash
python3 -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
python -m ipykernel install --user --name "adk-env" --display-name "Python (adk-env)"
```
- **select the kernel**
  In Jupyter: Kernel → Change kernel → Python (adk-env)

## Add robust retry/backoff (prevents 429 crashes)

In [ ]:
import time, random

def with_backoff(max_retries=6, base=1.0, jitter=0.25, retry_on=("RESOURCE_EXHAUSTED", "429", "503")):
    def wrap(fn):
        def inner(*args, **kwargs):
            delay = base
            for attempt in range(1, max_retries+1):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    s = str(e)
                    if any(code in s for code in retry_on):
                        wait = delay + random.uniform(0, jitter)
                        print(f"⚠️ {s[:80]}…  retry {attempt}/{max_retries} in {wait:.2f}s")
                        time.sleep(wait); delay *= 2
                        continue
                    raise
            raise RuntimeError(f"Failed after {max_retries} retries.")
        return inner
    return wrap


## Minimal model call (smoke test)


In [ ]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

@with_backoff()
def ping_model(prompt="Say 'ready'.", model_name="gemini-1.5-flash"):
    model = genai.GenerativeModel(model_name, generation_config={"max_output_tokens": 64})
    return model.generate_content(prompt).text

print("🔎 Model says:", ping_model())


In [2]:
##🎛️ tune a script for one comaand set up 
- setup_env.sh
```bash

#!/usr/bin/env bash
python3 -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
python -m ipykernel install --user --name "adk-env" --display-name "Python (adk-env)"
echo "Done. Start Jupyter and select kernel: Python (adk-env)"
```